In [2]:
#Create text file with links (EXECUTE THE NEXT 2 CELLS ONLY ONCE)
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

anime = []

for page in tqdm(range(0, 400)):
    url = 'https://myanimelist.net/topanime.php?limit=' + str(page * 50)
    response = requests.get(url)
    
    soup = BeautifulSoup(response.text, 'html.parser')
    for tag in soup.find_all('tr'):
        links = tag.find_all('a')
        for link in links:        
            if type(link.get('id')) == str and len(link.contents[0]) > 1:
                anime.append((link.contents[0], link.get('href')) )


100%|██████████| 400/400 [05:13<00:00,  1.27it/s]


In [3]:
f = open("links.txt", "w")
for el in anime:
    f.write(str(el[1])+'\n')
f.close()
#We get 19083 rows, which implies we have less than 20000 animes

In [ ]:
#Download the pages
#import time
import os
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from random import randint
#multiprocessing library

#first open file from which we get links
f = open("links.txt","r")
lines = f.readlines()
#seconds=60

for page in tqdm(range(100,105)):
    os.system("mkdir Page"+str(page+1)) #the (page+1) is to make so we don't have the first element 0 but 1
    os.chdir("Page"+str(page+1))
    
    if page%2==0: start=page*50
    if page%2!=0: start=page*50+50
    
    for i in range(start,start+50):
        url=lines[i]
        check = requests.get(url)
        
        #Check if the site is blocking the download
        if check.status_code != 200:
            print("Last well executed page is {}".format(page-1))
            while(check.status_code != 200): #Stop while we can't download
                check = requests.get(url)
                #os.system(("sudo ifconfig lo 127.0.0.{}  netmask 255.0.0.0").format(randint(0,50)))
       
          
        art=open("article"+str(i+1)+'.html',"wb")
        art.write(check.content)
        art.close()
    
    os.chdir("..")

  0%|          | 0/5 [00:00<?, ?it/s]

Last well executed page is 99
